# Lab_08, Logika rozmyta

## Zad 01

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Zmienne lingwistyczne
quality = ctrl.Antecedent(np.arange(0, 11, 1), "quality")
service = ctrl.Antecedent(np.arange(0, 11, 1), "service")
tip = ctrl.Consequent(np.arange(0, 26, 1), "tip")

quality.automf(3)
service.automf(3)

tip["low"] = fuzz.trimf(tip.universe, [0, 0, 13])
tip["medium"] = fuzz.trimf(tip.universe, [0, 13, 25])
tip["high"] = fuzz.trimf(tip.universe, [13, 25, 25])

quality["average"].view()
service.view()
tip.view()

In [ ]:
# reguły

rule1 = ctrl.Rule(quality["poor"] | service["poor"], tip["low"])
rule2 = ctrl.Rule(service["average"], tip["medium"])
rule3 = ctrl.Rule(service["good"] | quality["good"], tip["high"])

rule1.view()

In [ ]:
tipping_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
tipping = ctrl.ControlSystemSimulation(tipping_ctrl)

tipping.input["quality"] = 6.5
tipping.input["service"] = 9.8

tipping.compute()

print(f"Tipping: {tipping.output['tip']}")
tip.view(sim=tipping)

## Zad 02

In [ ]:
# d) wczytanie modelu z pliku
import os
import shutil
from gan.gan07 import train, define_discriminator, define_gan, load_real_samples
from keras.models import load_model

g_model = load_model("gan/generator_model_005.h5")
d_model = define_discriminator()
letent_dim = 100
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
n_epochs = 5
batch_size = 256

train(g_model, d_model, gan_model, dataset, letent_dim, n_epochs, batch_size)

# move all generated files to "generated" folder
if not os.path.exists("generated"):
    os.makedirs("generated")

for file in os.listdir("."):
    if file.endswith(".png"):
        shutil.move(file, "generated")
    if file.endswith(".h5"):
        shutil.move(file, "generated")

In [ ]:
# e) train model with bigger number of epochs (50)
n_epochs = 50
train(g_model, d_model, gan_model, dataset, letent_dim, n_epochs, batch_size)

## Zad 03

In [ ]:
%run rnn/lstm01.py

In [ ]:
%run rnn/lstm02.py

In [ ]:
%run rnn/lstm03.py

In [ ]:
%run rnn/lstm04.py

In [ ]:
%run rnn/rnn01.py

In [ ]:
%run rnn/rnn02.py

In [ ]:
%run rnn/rnn03.py

## Zadanie 04

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

# stable-diffusion-v1-5
# wytrenowany na obrazach 512x512 z bazy LAION-5B
# Hardware: 32 x 8 x A100 GPUs
# Optimizer: AdamW
# Gradient Accumulations: 2
# Batch: 32 x 8 x 2 x 4 = 2048 
# Learning rate: warmup to 0.0001 for 10,000 steps and then kept constant
# Hardware Type: A100 PCIe 40GB
# Wagi: 4,72GB

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]

image.save("astronaut_rides_horse.png")